# A New QR Algorithm

The step we use is given by $$A(nb+1:n,nb+1:n)\leftarrow A(nb+1:n, nb+1:n)-A(nb+1:n,1:nb)\times A(1:nb,nb+1:n)$$

In [3]:
using LinearAlgebra
I(3)[2,:]

3-element Vector{Bool}:
 0
 1
 0

In [186]:
#so in code form we find
function NewHouseholderQR(A)
    n,_ = size(A)
    Tau = zeros(ComplexF64, n)
    for nb = 1:n
        Tau[nb] = (1+abs(A[nb,nb]))
        A[nb+1:n, nb] = A[nb+1:n, nb] / (A[nb, nb] + (A[nb, nb] / abs(A[nb, nb])))
        A[nb, nb] = -(A[nb, nb]/abs(A[nb, nb]))
        A[nb+1:n,nb+1:n]=A[nb+1:n, nb+1:n]-A[nb+1:n,1:nb]*A[1:nb, nb+1:n]
    end
    A, Tau
end

function GetR(A, Tau)
    diagm(0 => diag(A, 0))
end

function GetQ(A, Tau)
    n, _ = size(A)
    Q = diagm(ones(ComplexF64, n))
    for i in n:-1:1
        u = A[i:n, i]
        u[1] = 1
        Q[i:n, :] = Q[i:n, :] - Tau[i] * u * (u' * Q[i:n, :])
    end
    Q
end

GetQ (generic function with 1 method)

In [ ]:
using LinearAlgebra

function GenerateRandUnitaries(n)
    [Matrix(qr(rand(ComplexF64, (2^i,2^i))).Q) for i=1:n]
end

n = 3
U = Matrix(qr(rand(ComplexF64, (2^n, 2^n))).Q)
UCopy = deepcopy(U)
OurHH = NewHouseholderQR(U)
OurR = GetR(OurHH[1], OurHH[2])
OurQ = GetQ(OurHH[1], OurHH[2])
Res = (OurQ * OurR)
Res ≈ UCopy

8×8 Matrix{ComplexF64}:
 0.694762+0.719239im    0.0981941+0.117214im   …   -0.244568-0.141231im
  0.28036+0.0915878im    0.999943-0.0107125im       0.387478-0.431588im
 0.208102+0.19558im     0.0676689+0.11434im       -0.0930261+0.0165807im
 0.138687+0.0643109im    0.153522+0.305871im       -0.568872+0.0668973im
  0.25937-0.142499im     0.118581+0.365482im      -0.0148404-0.262152im
 0.245354-0.0883302im   -0.273544+0.108924im   …    0.654548-0.618532im
 0.202784+0.0985112im  -0.0286135-0.196491im         1.19129+1.31397im
 0.352301+0.0180443im   0.0511395-0.082095im      0.00981808-0.999952im

true